<center><h1><b>Prédiction et Optimisation de la Maintenance Aéronautique</b></h1></center>


* Script pour récupérer automatiquement ces fichiers chaque jour
* Mettre à jour les datasets composants et aéronefs selon les nouvelles données
* Analyser et visualiser l’évolution des dégradations et états des voyants
* Identifier des corrélations entre variables pour prédire les voyants
* Construire un modèle prédisant les voyants après chaque vol
* Pour chaque vol, estimer le coût d'immobilisation selon le voyant
* Comparer les coûts totaux dans différents scénarios
* Optimiser le modèle pour maximiser les économies en maintenance
* Proposer des axes de développement futur

In [11]:
import pandas as pd
from sqlalchemy import create_engine

In [12]:
# Configuration de la connexion à la base de données MySQL
db_user = 'Top_gun'
db_password = 'zg6N&284Bb<w'
db_host = '212.227.48.180'
db_port = '3306'
db_name = 'Top_gun'

# Création du moteur de connexion
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [13]:
aeronefs = 'aeronefs'
df_aeronefs = pd.read_sql_table(aeronefs, engine)

composants = 'composants'
df_composants = pd.read_sql_table(composants, engine)

vols = 'logs_vols'
df_logs_vols = pd.read_sql_table(vols, engine)

degradations = 'degradations'
df_degradations = pd.read_sql_table(degradations, engine)

df_logs_vols['jour_vol'] = pd.to_datetime(df_logs_vols['jour_vol'])
df_degradations['measure_day'] = pd.to_datetime(df_degradations['measure_day'])

In [14]:
df_aeronefs.head()

,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
0,E170_6353,E170,2000-03-17,2024-03-27 00:00:00,0,None
1,B767_2803,B767,2020-12-03,2024-06-01 00:00:00,0,None
2,CRJ700_6943,CRJ700,2015-01-29,2024-06-02,1,2024-06-03 14:13:00
3,A350_1177,A350,2005-08-07,2024-01-04 00:00:00,0,None
4,B777_4029,B777,2009-01-01,2024-06-01 00:00:00,0,None


In [15]:
df_composants.head()

,ref_compo,categorie,aero,desc,lifespan,taux_usure_actuel,cout
0,REAE170-E170_6353-0,Composants Critiques,E170_6353,Réacteur gauche,13045,53.308375,15552
1,REAE170-E170_6353-1,Composants Critiques,E170_6353,Réacteur droit,13626,50.111950,17559
2,SYSE170-E170_6353-2,Composants Critiques,E170_6353,Système de navigation,10541,16.730575,16514
3,ORDE170-E170_6353-3,Composants Critiques,E170_6353,Ordinateur de vol,10441,6.653075,16047
4,AUTE170-E170_6353-4,Composants Critiques,E170_6353,Autopilote,14847,42.033525,19362


In [16]:
df_logs_vols.head()

,ref_vol,aero_linked,jour_vol,time_en_air,etat_voyant,temp en °C,pressure en hPa,vibrations en m/s²
0,V07679725,B767_4924,2024-06-02,7.0,0,-5.7,930.6,3.116391
1,V02564879,B747_3165,2024-06-02,5.1,0,0.9,950.6,4.557825
2,V07933973,B757_2518,2024-06-02,7.7,3,-6.2,975.5,4.587071
3,V02230187,A340_1047,2024-06-02,6.1,1,-25.3,945.1,3.287869
4,V09071213,B757_6806,2024-06-02,4.5,0,11.1,931.3,2.816626


In [17]:
df_degradations.head()

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,74.0,2024-07-22,0
1,D009402,E170_6353,REAE170-E170_6353-1,72.0,2024-07-22,0
2,D000812,E170_6353,SYSE170-E170_6353-2,34.0,2024-07-22,0
3,D006665,E170_6353,ORDE170-E170_6353-3,24.0,2024-07-22,0
4,D001163,E170_6353,AUTE170-E170_6353-4,66.0,2024-07-22,0


In [28]:
# Lignes dupliquées pour chaque DataFrame
dataframes = {
    'df_degradations': df_degradations,
    'df_logs_vols': df_logs_vols,
    'df_aeronefs': df_aeronefs,
    'df_composants': df_composants
}

for name, df in dataframes.items():
    duplicated_count = df.duplicated().sum()
    total_count = len(df)
    unique_count = total_count - duplicated_count
    print(f"Nombre de valeurs uniques pour la table {name}: {unique_count}")
    print(f"Nombre de lignes dupliquées pour la table {name}: {duplicated_count}")
    print("__________________________________________________________________")

Nombre de valeurs uniques pour la table df_degradations: 241470
Nombre de lignes dupliquées pour la table df_degradations: 0
__________________________________________________________________
Nombre de valeurs uniques pour la table df_logs_vols: 5315
Nombre de lignes dupliquées pour la table df_logs_vols: 0
__________________________________________________________________
Nombre de valeurs uniques pour la table df_aeronefs: 232
Nombre de lignes dupliquées pour la table df_aeronefs: 0
__________________________________________________________________
Nombre de valeurs uniques pour la table df_composants: 10440
Nombre de lignes dupliquées pour la table df_composants: 0
__________________________________________________________________


In [30]:
# Dictionnaire des DataFrames
dataframes = {
    'df_degradations': df_degradations,
    'df_logs_vols': df_logs_vols,
    'df_aeronefs': df_aeronefs,
    'df_composants': df_composants
}

for name, df in dataframes.items():
    null_count = df.isnull().sum().sum()  # Nombre total de valeurs nulles
    
    print(f"Table: {name}")
    print(f"Nombre total de valeurs nulles: {null_count}")
    print("__________________________________________________________________")


Table: df_degradations
Nombre total de valeurs nulles: 270
__________________________________________________________________
Table: df_logs_vols
Nombre total de valeurs nulles: 0
__________________________________________________________________
Table: df_aeronefs
Nombre total de valeurs nulles: 217
__________________________________________________________________
Table: df_composants
Nombre total de valeurs nulles: 0
__________________________________________________________________


In [19]:
# # Mettre à jour la colonne "need_replacement" à 1 pour les lignes où "usure_nouvelle" est >= 75%
# if isinstance(df_degradations, pd.DataFrame):
#     df_degradations.loc[df_degradations["usure_nouvelle"] >= 75, "need_replacement"] = 1


# df_degradations

In [20]:
type(df_degradations)

pandas.core.frame.DataFrame